### Prepare datasets for analysis

## Lending Club Analysis

Import basic libraries

In [1]:
import numpy as np
import pandas as pd

Change pandas print options so we can print all desired rows/columns without truncation

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

Read in the data

In [3]:
acc_df = pd.read_csv(
    '../input/accepted_small.csv.gz',
    low_memory=True,
    nrows=1000,
    delimiter=';'
)

In [4]:
rej_df = pd.read_csv(
    '../input/rejected_small.csv.gz',
    compression='gzip',
    low_memory=True,
    nrows=1000,
    delimiter=';'
)

### Have a first look

In [5]:
acc_df.shape, rej_df.shape

((1000, 8), (1000, 8))

In [6]:
acc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
9eca31d1-d5e7-4344-a0e5-da06bd8564d9    1000 non-null object
1                                       1000 non-null int64
loan_amnt                               1000 non-null float64
title                                   999 non-null object
emp_length                              959 non-null object
annual_inc                              1000 non-null object
debt_to_income_ratio                    1000 non-null object
term                                    1000 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 62.6+ KB


In [7]:
rej_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
1c0d8871-51b6-4224-a9a6-74b1fce4dc1d    1000 non-null object
0                                       1000 non-null int64
Amount Requested                        1000 non-null float64
Loan Title                              1000 non-null object
Employment Length                       951 non-null object
annual_inc                              1000 non-null object
Debt-To-Income Ratio                    1000 non-null object
term                                    1000 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 62.6+ KB


In [8]:
acc_df.head(3)

,9eca31d1-d5e7-4344-a0e5-da06bd8564d9,1,loan_amnt,title,emp_length,annual_inc,debt_to_income_ratio,term
0,4f3f132a-2469-4231-9ccf-84180584f52d,1,15000.0,Debt consolidation,10+ years,78000.0,debt_to_income_ratio,60 months
1,309bf18d-9330-42eb-9214-060c80b697a6,1,10400.0,Credit card refinancing,8 years,58000.0,debt_to_income_ratio,36 months
2,3e48d9d1-94ec-4ffa-8ccd-3e86b84ae156,1,21425.0,Credit card refinancing,6 years,63800.0,debt_to_income_ratio,60 months


In [9]:
rej_df.head(3)

,1c0d8871-51b6-4224-a9a6-74b1fce4dc1d,0,Amount Requested,Loan Title,Employment Length,annual_inc,Debt-To-Income Ratio,term
0,2ad6224b-718f-4d1a-922b-aae668213b79,0,4000.0,major_purchase,5 years,annual_inc,4.21%,term
1,6a6669ee-d6a7-4c1f-9db2-7494d2990977,0,20000.0,debt_consolidation,< 1 year,annual_inc,0.39%,term
2,27c963fc-f84f-41bf-a7d3-9ffc9fc49906,0,1000.0,renewable_energy,5 years,annual_inc,42.38%,term


In [10]:
acc_df.describe()

,1,loan_amnt
count,1000.0,1000.000000
mean,1.0,14216.175000
std,0.0,8648.300243
min,1.0,1000.000000
25%,1.0,7368.750000
50%,1.0,12000.000000
75%,1.0,20000.000000
max,1.0,35000.000000


In [11]:
rej_df.describe()

,0,Amount Requested
count,1000.0,1000.000000
mean,0.0,11483.550000
std,0.0,14778.887133
min,0.0,1000.000000
25%,0.0,3000.000000
50%,0.0,6000.000000
75%,0.0,15075.000000
max,0.0,300000.000000


In [12]:
categorical_features = []
for column in acc_df.columns:
    if acc_df[column].dtype == np.object:
        categorical_features.append(column)

acc_df[categorical_features].describe()

,9eca31d1-d5e7-4344-a0e5-da06bd8564d9,title,emp_length,annual_inc,debt_to_income_ratio,term
count,1000,999,959,1000,1000,1000
unique,1000,15,25,297,1,2
top,c79ccdbd-4d5b-4e8a-ab4e-a010677ce1c9,Debt consolidation,10+ years,60000.0,debt_to_income_ratio,36 months
freq,1,557,334,41,1000,723


In [13]:
categorical_features = []
for column in rej_df.columns:
    if rej_df[column].dtype == np.object:
        categorical_features.append(column)

rej_df[categorical_features].describe()

,1c0d8871-51b6-4224-a9a6-74b1fce4dc1d,Loan Title,Employment Length,annual_inc,Debt-To-Income Ratio,term
count,1000,1000,951,1000,1000,1000
unique,1000,21,9,1,830,1
top,b7cbb64b-6086-4ffa-beb7-8f8d4682f980,debt_consolidation,< 1 year,annual_inc,-1%,term
freq,1,470,724,1000,53,1000


### What both datasets have in common?

In [14]:
rej_df.columns

Index(['1c0d8871-51b6-4224-a9a6-74b1fce4dc1d', '0', 'Amount Requested',
       'Loan Title', 'Employment Length', 'annual_inc', 'Debt-To-Income Ratio',
       'term'],
      dtype='object')

### Amount Requested

In [15]:
acc_df[acc_df.columns[acc_df.columns.where(acc_df.columns.map(lambda x: 'amnt' in x or 'Amount' in x)).notna()]].head(5)

,loan_amnt
0,15000.0
1,10400.0
2,21425.0
3,12800.0
4,7650.0


Corresponding features: `acc_df['loan_amnt'] === rej_df['Amount Requested']`

### Loan Title

In [16]:
acc_df[acc_df.columns[acc_df.columns.where(acc_df.columns.map(lambda x: 'loan' in x or 'Loan' in x)).notna()]].describe()

,loan_amnt
count,1000.000000
mean,14216.175000
std,8648.300243
min,1000.000000
25%,7368.750000
50%,12000.000000
75%,20000.000000
max,35000.000000


In [17]:
acc_df[acc_df.columns[acc_df.columns.where(acc_df.columns.map(lambda x: 'title' in x or 'Title' in x)).notna()]].describe()

,title
count,999
unique,15
top,Debt consolidation
freq,557


In [18]:
rej_df['Loan Title'].describe()

count                   1000
unique                    21
top       debt_consolidation
freq                     470
Name: Loan Title, dtype: object

Corresponding features: `acc_df['title'] === rej_df['Loan Title']`

### Debt-To-Income Ratio

In [19]:
rej_df['Debt-To-Income Ratio'].describe()

count     1000
unique     830
top        -1%
freq        53
Name: Debt-To-Income Ratio, dtype: object

In [20]:
rej_df['Debt-To-Income Ratio'] = rej_df['Debt-To-Income Ratio'].str.extract('([-]?[0123456789\.]+)', expand=True).astype(float)

In [21]:
rej_df['Debt-To-Income Ratio'].describe()

count     1000.000000
mean       183.392810
std       3251.644334
min         -1.000000
25%          7.045000
50%         18.935000
75%         34.700000
max      99999.000000
Name: Debt-To-Income Ratio, dtype: float64

In [22]:
acc_df[acc_df.columns[acc_df.columns.where(acc_df.columns.map(lambda x: 'inc' in x or 'Inc' in x)).notna()]].describe()

,annual_inc,debt_to_income_ratio
count,1000,1000
unique,297,1
top,60000.0,debt_to_income_ratio
freq,41,1000


Corresponding features (different measure units): `acc_df['annual_inc'] === rej_df['Debt-To-Income Ratio']`

### Employment Length

In [23]:
rej_df[rej_df['Employment Length'].notnull()]['Employment Length'].unique()

array(['5 years', '< 1 year', '4 years', '6 years', '3 years', '1 year',
       '10+ years', '9 years', '2 years'], dtype=object)

In [24]:
acc_df['emp_length'].unique()

array(['10+ years', '8 years', '6 years', '< 1 year', '2 years',
       '9 years', '7 years', '5 years', '3 years', '1 year', nan,
       '4 years', ' Information Technology"', ' Acad"', ' Regulatory"',
       ' Senior Director"', ' Finance"', ' Planning',
       ' Relationship Mgmt."', ' HUMAN RESOURCES"', ' Engineering"',
       ' Team Leader"', ' CEO"', ' Lexus Production"', ' varsity coach"',
       ' Technology"'], dtype=object)

Corresponding features: `acc_df['emp_length'] === rej_df['Employment Length']`

### Term

In [25]:
acc_df['term'].unique()

array([' 60 months', ' 36 months'], dtype=object)